In this example, we will build a Telegram bot that sends a signal once any Bollinger Band has been crossed. We will periodically query for the latest OHLCV data of the selected cryptocurrencies and append this data to our data pool. Additionally to receiving signals, any Telegram user can join the group and ask the bot to provide him with the current information. If the price change is higher than some number of standard deviations from the mean, while crossing the band, the bot sends a funny GIF.

In [1]:
from telegram import __version__ as TG_VER

try:
    from telegram import __version_info__
except ImportError:
    __version_info__ = (0, 0, 0, 0, 0)

if __version_info__ >= (20, 0, 0, "alpha", 1):
    raise RuntimeError(f"This example is not compatible with your current PTB version {TG_VER}")

In [2]:
# whats_imported()
import logging

from vectorbtpro import *

In [3]:
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

In [4]:
# Telegram
vbt.settings.telegram["bot"]["token"] = "YOUR_TOKEN"

# Giphy
vbt.settings.telegram["giphy"]["api_key"] = "YOUR_API_KEY"

# Data
SYMBOLS = ["BTC/USDT", "ETH/USDT", "ADA/USDT"]
START = "1 hour ago UTC"
TIMEFRAME = "1m"
UPDATE_EVERY = vbt.utils.datetime_.interval_to_ms(TIMEFRAME) // 1000  # in seconds
DT_FORMAT = "%d %b %Y %H:%M:%S %z"
IND_PARAMS = dict(timeperiod=20, nbdevup=2, nbdevdn=2)
CHANGE_NBDEV = 2

In [5]:
data = vbt.CCXTData.pull(SYMBOLS, start=START, timeframe=TIMEFRAME)

print(data.wrapper.index)

  0%|          | 0/3 [00:04<?, ?it/s, symbol=BTC/USDT]

1it [00:04,  4.22s/it, 2024-09-24 11:18:25.737 → 2024-09-24 12:18]

DatetimeIndex(['2024-09-24 11:19:00+00:00', '2024-09-24 11:20:00+00:00',
               '2024-09-24 11:21:00+00:00', '2024-09-24 11:22:00+00:00',
               '2024-09-24 11:23:00+00:00', '2024-09-24 11:24:00+00:00',
               '2024-09-24 11:25:00+00:00', '2024-09-24 11:26:00+00:00',
               '2024-09-24 11:27:00+00:00', '2024-09-24 11:28:00+00:00',
               '2024-09-24 11:29:00+00:00', '2024-09-24 11:30:00+00:00',
               '2024-09-24 11:31:00+00:00', '2024-09-24 11:32:00+00:00',
               '2024-09-24 11:33:00+00:00', '2024-09-24 11:34:00+00:00',
               '2024-09-24 11:35:00+00:00', '2024-09-24 11:36:00+00:00',
               '2024-09-24 11:37:00+00:00', '2024-09-24 11:38:00+00:00',
               '2024-09-24 11:39:00+00:00', '2024-09-24 11:40:00+00:00',
               '2024-09-24 11:41:00+00:00', '2024-09-24 11:42:00+00:00',
               '2024-09-24 11:43:00+00:00', '2024-09-24 11:44:00+00:00',
               '2024-09-24 11:45:00+00:00', '2024-0

In [6]:
def get_bbands(data):
    return vbt.IndicatorFactory.from_talib("BBANDS").run(
        data.get("Close"), **IND_PARAMS, hide_params=list(IND_PARAMS.keys())
    )


def get_info(bbands):
    info = dict()
    info["last_price"] = bbands.close.iloc[-1]
    info["last_change"] = (bbands.close.iloc[-1] - bbands.close.iloc[-2]) / bbands.close.iloc[-1]
    info["last_crossed_above_upper"] = bbands.close_crossed_above(bbands.upperband).iloc[-1]
    info["last_crossed_below_upper"] = bbands.close_crossed_below(bbands.upperband).iloc[-1]
    info["last_crossed_below_lower"] = bbands.close_crossed_below(bbands.lowerband).iloc[-1]
    info["last_crossed_above_lower"] = bbands.close_crossed_above(bbands.lowerband).iloc[-1]
    info["bw"] = (bbands.upperband - bbands.lowerband) / bbands.middleband
    info["last_bw_zscore"] = info["bw"].vbt.zscore().iloc[-1]
    info["last_change_zscore"] = bbands.close.vbt.pct_change().vbt.zscore().iloc[-1]
    info["last_change_pos"] = info["last_change_zscore"] >= CHANGE_NBDEV
    info["last_change_neg"] = info["last_change_zscore"] <= -CHANGE_NBDEV
    return info


def format_symbol_info(symbol, info):
    last_change = info["last_change"][symbol]
    last_price = info["last_price"][symbol]
    last_bw_zscore = info["last_bw_zscore"][symbol]
    return f"{symbol} ({last_change:.2%}, {last_price}, {last_bw_zscore:.2f})"


def format_signals_info(emoji, signals, info):
    symbols = signals.index[signals]
    symbol_msgs = []
    for symbol in symbols:
        symbol_msgs.append(format_symbol_info(symbol, info))
    return "{} {}".format(emoji, ", ".join(symbol_msgs))

In [8]:
from telegram.ext import CommandHandler


class MyTelegramBot(vbt.TelegramBot):
    def __init__(self, data, **kwargs):
        super().__init__(data=data, **kwargs)

        self.data = data
        self.update_ts = data.wrapper.index[-1]

    @property
    def custom_handlers(self):
        return (CommandHandler("info", self.info_callback),)

    def info_callback(self, update, context):
        chat_id = update.effective_chat.id
        if len(context.args) != 1:
            self.send_message(chat_id, "Please provide one symbol.")
            return
        symbol = context.args[0]
        if symbol not in SYMBOLS:
            self.send_message(chat_id, f'There is no such symbol as "{symbol}".')
            return

        bbands = get_bbands(self.data)
        info = get_info(bbands)
        messages = [format_symbol_info(symbol, info)]
        message = "\n".join([f"{self.update_ts.strftime(DT_FORMAT)}:"] + messages)
        self.send_message(chat_id, message)

    @property
    def start_message(self):
        index = self.data.wrapper.index
        return f"""Hello! 

Starting with {len(index)} rows from {index[0].strftime(DT_FORMAT)} to {index[-1].strftime(DT_FORMAT)}."""

    @property
    def help_message(self):
        return """Message format:
[event] [symbol] ([price change], [new price], [bandwidth z-score])
    
Event legend:
⬆️ - Price went above upper band
⤵️ - Price retraced below upper band
⬇️ - Price went below lower band
⤴️ - Price retraced above lower band

GIF is sent once a band is crossed and the price change is 2 stds from the mean."""

In [9]:
telegram_bot = MyTelegramBot(data)
telegram_bot.start(in_background=True)

2024-09-24 14:18:47,477 - vectorbtpro.utils.telegram - INFO - Initializing bot
2024-09-24 14:18:47,482 - vectorbtpro.utils.telegram - INFO - Loaded chat ids [447924619]
2024-09-24 14:18:47,703 - vectorbtpro.utils.telegram - INFO - Running bot vectorbt_bot
2024-09-24 14:18:47,705 - apscheduler.scheduler - INFO - Scheduler started
2024-09-24 14:18:48,076 - vectorbtpro.utils.telegram - INFO - 447924619 - Bot: "I'm back online!"
2024-09-24 14:19:09,843 - vectorbtpro.utils.telegram - INFO - 447924619 - User: "/info"
2024-09-24 14:19:09,959 - vectorbtpro.utils.telegram - INFO - 447924619 - Bot: "Please provide one symbol."
2024-09-24 14:19:26,798 - vectorbtpro.utils.telegram - INFO - 447924619 - User: "/info BTC/USDT"
2024-09-24 14:19:27,182 - vectorbtpro.utils.telegram - INFO - 447924619 - Bot: "24 Sep 2024 12:18:00 +0000:
BTC/USDT (-0.03%, 63553.99, -0.07)"


In [10]:
class MyDataUpdater(vbt.DataUpdater):
    _expected_keys = None

    def __init__(self, data, telegram_bot, **kwargs):
        super().__init__(data, telegram_bot=telegram_bot, **kwargs)

        self.telegram_bot = telegram_bot
        self.update_ts = data.wrapper.index[-1]

    def update(self):
        super().update()
        self.update_ts = vbt.timestamp(tz=self.update_ts.tz)
        self.telegram_bot.data = self.data
        self.telegram_bot.update_ts = self.update_ts

        bbands = get_bbands(self.data)
        info = get_info(bbands)

        messages = []
        if info["last_crossed_above_upper"].any():
            messages.append(format_signals_info("⬆️", info["last_crossed_above_upper"], info))
        if info["last_crossed_below_upper"].any():
            messages.append(format_signals_info("⤵️", info["last_crossed_below_upper"], info))
        if info["last_crossed_below_lower"].any():
            messages.append(format_signals_info("⬇️", info["last_crossed_below_lower"], info))
        if info["last_crossed_above_lower"].any():
            messages.append(format_signals_info("⤴️", info["last_crossed_above_lower"], info))

        if len(messages) > 0:
            message = "\n".join([f"{self.update_ts.strftime(DT_FORMAT)}:"] + messages)
            self.telegram_bot.send_message_to_all(message)
        if (info["last_crossed_above_upper"] & info["last_change_pos"]).any():
            self.telegram_bot.send_giphy_to_all("launch")
        if (info["last_crossed_below_lower"] & info["last_change_neg"]).any():
            self.telegram_bot.send_giphy_to_all("fall")

In [11]:
data_updater = MyDataUpdater(data, telegram_bot)
data_updater.update_every(UPDATE_EVERY)

2024-09-24 14:18:53,111 - vectorbtpro.utils.schedule_ - INFO - Starting schedule manager with jobs [Every 60 seconds do update() (last run: [never], next run: 2024-09-24 14:19:53)]
/Users/olegpolakow/Documents/GitHub/vectorbt.pro/vectorbtpro/data/base.py:3335: UserWarning: Symbols have mismatching index. Setting missing data points to NaN.
  new_data = self.align_index(new_data, missing=self.missing_index, silence_warnings=silence_warnings)
2024-09-24 14:20:09,757 - vectorbtpro.data.updater - INFO - New data has 62 rows from 2024-09-24 11:19:00+00:00 to 2024-09-24 12:20:00+00:00
2024-09-24 14:20:09,928 - vectorbtpro.utils.telegram - INFO - 447924619 - Bot: "24 Sep 2024 12:20:09 +0000:
⤵️ ADA/USDT (0.00%, 0.3707, -0.12)"
2024-09-24 14:20:38,758 - vectorbtpro.utils.schedule_ - INFO - Stopping schedule manager


In [12]:
telegram_bot.stop()

2024-09-24 14:20:42,046 - vectorbtpro.utils.telegram - INFO - Stopping bot
2024-09-24 14:20:42,048 - apscheduler.scheduler - INFO - Scheduler has been shut down
